### Processing pipeline for 'meta_amazon_instant_video.json' 

<font color='royalblue' size=3>
<b>

Parameters for the notebook

In [95]:
datapath = 'DATA/'
df_name = "meta_Amazon_Instant_Video"


###
###  Change here which fields will be extracted from the amazon product
###
def set_df_cell_from_product(amazonProduct, rowNumber, fieldName1, fieldName2) :
    if (amazonProduct is None) : 
        amazon_products_df.set_value(rowNumber, fieldName1, [])
        amazon_products_df.set_value(rowNumber, fieldName2, [])     
    else : 
        actors = amazonProduct.actors
        amazon_products_df.set_value(rowNumber, fieldName1, actors)
        directors = amazonProduct.directors
        amazon_products_df.set_value(rowNumber, fieldName2, directors)

# How frequently should we save our data to file
data_save_freq = 10000
# feature we're interested in
features = ['asin', 'categories']
json_name = df_name+'.json.gz'

<font color='royalblue' size=3>
<b>

Code

In [96]:
#essential imports
import pandas as pd
import numpy as np
import json

# Sleep
import time

# Strict JSON conversion
import json 
import gzip 

# Progress display
from IPython.display import clear_output

# Amazon API querying
from amazon.api import AmazonAPI
from amazon.api import AsinNotFound

from urllib.request import HTTPError


<font color='royalblue' size=3>
<b>

Open metadata file

In [97]:
## Load all ASINs we're going to query - use the metadata files
## for this, as they contains each ASIN once and only once.
##

''' This function was provided on the amazon dataset's webpage
    It loads a gzipped file directly into a dataframe
'''
def gz_to_dataframe(datapath, filename):
    def parse(path): 
        g = gzip.open(path, 'rb') 
        for l in g: 
            yield eval(l) 
    def getDF(path): 
        i = 0 
        df = {} 
        for d in parse(path): 
            df[i] = d 
            i += 1 
        return pd.DataFrame.from_dict(df, orient='index') 
    return getDF(datapath+filename)
    
amazon_products_df = gz_to_dataframe(datapath, json_name)
amazon_products_df.head(3)

,asin,categories,price,related
0,B000GFDAUG,[[Amazon Instant Video]],NaN,NaN
1,B000GIOPK2,[[Amazon Instant Video]],NaN,NaN
2,B000GIPKWY,[[Amazon Instant Video]],NaN,NaN


In [98]:
amazon_products_df = amazon_products_df[features]
amazon_products_df.head(3)

,asin,categories
0,B000GFDAUG,[[Amazon Instant Video]]
1,B000GIOPK2,[[Amazon Instant Video]]
2,B000GIPKWY,[[Amazon Instant Video]]


<font color='royalblue' size=3>
<b>

Prepare for amazon api usage

In [99]:
## Sign in with amazon API 
##

def get_amazon_interface():
    f = open("api_creds")
    ar = f.read().split("\n")
    return AmazonAPI(ar[0], ar[1], ar[2])
    return ar[0], ar[1], ar[2]

amazon = get_amazon_interface()

In [100]:
## Here, we define some API query helpers
##

''' Product lookup with API, asin can be a string ('one by one' lookup)
    or a list of strings ('bulk lookup').
    bulk lookup provides better performance
'''
def get_prod(asin) : 
    if not isinstance(asin, str): 
        acc_str = asin[0]
        for e in asin : 
            acc_str += ','+e
        print(acc_str)
        return amazon.lookup(ItemId=acc_str)
    else :
        return amazon.lookup(ItemId=asin)
    
''' Splits the interval [start-end] into bulks of size bulksize
'''    
def gen_bulk_index(start, end, bulksize=10, includeEnd=False):
    size = end - start + 1
    bulks = [list(range(start+(i*bulksize), start + (i+1)*bulksize)) for i in range(0, int(size/bulksize))]
    if includeEnd : 
        bulks.append(list(range(bulks[len(bulks)-1][bulksize-1], end+1)))
    else : 
        bulks.append(list(range(bulks[len(bulks)-1][bulksize-1], end)))
    return bulks    


def get_directors(prod) : 
    return prod.directors

def get_actors(prod) : 
    return prod.actors


In [101]:
## Also, we have some functions to save the state of our data structure - in case we need to 
## to shutdown the computer and restart the query loop at a later time (not used here)
##

def save_progress(dataframe, nb_rows_processed):
    dataframe.to_csv(datapath+df_name+"_temp.csv")
    file = open(datapath+df_name+"_progress", "w")
    file.write(str(nb_rows_processed))

    
def load_progress():
    dataframe = pd.read_csv(datapath+df_name+"_temp.csv")
    file = open(datapath+df_name+"_progress", "r")
    nb_rows_processed = file.readline()
    return dataframe, int(nb_rows_processed)


<font color='royalblue' size=3>
<b>

Bulk lookups parameters and loop

In [102]:
## Parameters & initialization for bulk item lookup
##

bulksize = 10

# Change this to restore progress from file
fresh_run = False

if fresh_run : 
    # used to restart from where we were in case of unexepected network error
    lastItemLookedUp = 0
    amazon_products_df['actors'] = pd.Series(dtype=object)
    amazon_products_df['directors'] = pd.Series(dtype=object)
else : 
    amazon_products_df, lastItemLookedUp = load_progress()
    
print("last item looked up : ", lastItemLookedUp,  "  -  time : ",time.strftime("%H:%M:%S"), "\n\n")

last item looked up :  2829   -  time :  17:53:21 




In [103]:
## Querying loop
##

ref_for_progress = lastItemLookedUp
caught_httperrors = 0
loop_complete = False


while not loop_complete : 
    try : 
        
        for bulk in gen_bulk_index(lastItemLookedUp, amazon_products_df.shape[0], bulksize=bulksize) : 
            # update progess every 100 items
            if ((bulk[0]-(ref_for_progress)) % 100 == 0) : 
                clear_output()
                print("    ",int(100 * (bulk[0]+1) / amazon_products_df.shape[0]), "% completed (",bulk[0], " rows)", "  -  time : ",time.strftime("%H:%M:%S"))
                print("         Last Item Looked up : ", lastItemLookedUp, " / ", amazon_products_df.shape[0])
                print("         HTTPErrors caught : ", caught_httperrors)
                print("\n\n\n")

            # get asins for the bulk and fetch the matching AmazonProducts
            asins = amazon_products_df['asin'][bulk].tolist()
            prods = get_prod(asins)

            # Then, process each product to add necessary informations in the dataframe
            if (type(prods) is list) and (len(prods) == bulksize) :              
                # Case : we found exactly one result per ASIN
                #        process by bulk
                    for i, prod in enumerate(prods) : 
                        set_df_cell_from_product(prod, bulk[i], "actors", "directors")
            elif (type(prods) is list) or (type(prods) is AmazonApi.AmazonProduct) :  
                # Case : we obtained a list of AmazonProducts or a single AmazonProduct
                #        fallback to 1-by-1 querying
                for n in bulk :               
                    asin = amazon_products_df['asin'][n]
                    try : 
                        prod = get_prod(asin)
                    except(AsinNotFound): 
                        prod = None
                    set_df_cell_from_product(prod, n, "actors", "directors")
                    time.sleep(0.15)

            # Save progress
            lastItemLookedUp = bulk[bulksize-1]
            
            # Save data to file according to specified frequency
            if ((bulk[0]-(original_lastItemLookedUp)) % data_save_freq == 0) : 
                save_progress(amazon_products_df, lastItemLookedUp)
            
            # limit query frequency to avoid 503 errors
            time.sleep(min(bulksize/10, 5))
            
        loop_complete = True
        

    except HTTPError :
        # Nothing to do, loop_complete is still false,
        # We will go back to the query loop and start from the last recorded lastItemLookedUp
        
        # If we didn't make any progress, something must be wrong
        if lastItemLookedUp == original_lastItemLookedUp : 
            print("HTTPError caught at original_lastItemLookedUp  -  breaking")
            break 
        # else retry
        caught_httperrors += 1
        ref_for_progress = lastItemLookedUp
        
      
clear_output()    
save_progress(amazon_products_df, lastItemLookedUp)
print("amazon query loop completed !")
    

     11 % completed ( 3435  rows)   -  time :  17:55:26
         Last Item Looked up :  3435  /  30648
         HTTPErrors caught :  53




B002EC9NMI,B002EC9NMI,B002ECM3C0,B002EHRYKG,B002EJW0JO,B002EJSI5E,B002EKCWM8,B002ELJUCC,B002EP662U,B002ER3DYC,B002ESCRQ6


KeyboardInterrupt: 

In [ ]:
## Run this cell to manually save results
##

save_progress(amazon_products_df, lastItemLookedUp)

<font color='royalblue' size=3>
<b>

Save results

In [ ]:
amazon_products_df.to_csv(datapath+df_name+".csv")
del amazon_products_df  # free memory